# 🚀 DevOps Specialist - An AI Agent - CI/CD Pipeline Insight Engine with LLM + RAG

## 📖 Project Overview
This project builds a **Retrieval-Augmented Generation (RAG) system** using a combination of:
- **Vector Database (FAISS)** for semantic search  
- **Embedding Model** (`sentence-transformers/all-MiniLM-L6-v2`)  
- **Lightweight LLM** (`google/flan-t5-small`)

### The system is fine-tuned for **DevOps, CI/CD, Docker, and Kubernetes-related queries**.  
---

### 🛠 Steps Implemented

### ✅ 📂 1. Load & Import Libraries
Installed and imported the following libraries for initial serup
- numpy
- pandas
- matplotlib
- os
- pdfplumber
- nltk
- pathlib
- json
- openai
- tqdm
- transformers
- torch
- peft
- re
- datasets
- faiss
- pickle
- sentence_transformers

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import pdfplumber
import nltk
from pathlib import Path
import json
from openai import OpenAI
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, TrainingArguments, get_scheduler, BitsAndBytesConfig, LlamaTokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, AutoModel     
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from tqdm import tqdm
import re
from datasets import load_dataset
import faiss
import pickle
from sentence_transformers import SentenceTransformer

/opt/anaconda3/envs/spy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### ✅ 2. Data Preparation
- Collected knowledge in **JSONL format** with fields:  
  - `instruction` → user query (e.g., *What is CI?*)  
  - `input` → source reference (PDF section, abstract, etc.)  
  - `output` → final curated answer.  
- Decided to use **only the `output` field** for embeddings, so that the database directly stores **relevant knowledge answers**.

In [ ]:
PDF_DIR = "ci-cd-dataset"   # folder path with PDFs
CHUNK_SIZE = 400   # words per chunk

In [15]:
def chunk_text(text, chunk_size=CHUNK_SIZE):
    words = text.split()
    for i in range(0, len(words), chunk_size):
        yield " ".join(words[i:i+chunk_size])

In [16]:
pdf_chunks = {}  # {filename: [chunks]}
for pdf_file in Path(PDF_DIR).glob("*.pdf"):
    text = ""
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"

    chunks = list(chunk_text(text))
    pdf_chunks[pdf_file.name] = chunks
    print(f"{pdf_file.name}: {len(chunks)} chunks, {len(text.split())} words")

ci-cd_1.pdf: 7 chunks, 2411 words
ci-cd_2.pdf: 19 chunks, 7516 words
ci-cd_3.pdf: 11 chunks, 4047 words
ci-cd_7.pdf: 670 chunks, 267689 words


Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P2' is an invalid float value
Cannot set gray non-stroke color because /'P3' is an invalid float value
Cannot set gray non-stroke color because /'P3' is an invalid float value
Cannot set gray non-stroke color because /'P3' is an invalid float value
Cannot set gray non-stroke color because /'P3' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P4' is an invalid float value
Cannot set gray non-stroke color because /'P3' is an invalid float value
Cannot set gray non-stroke color because /'P3' is an invalid float value
Cannot set gray non-stroke color because /'P3' is an invalid float value
Cannot set gray non-stroke color because /'P3' is an invalid float value
Cannot set gray non-stroke color because /'P5' is a

ci-cd_6.pdf: 503 chunks, 200886 words
ci-cd_4.pdf: 19 chunks, 7584 words
ci-cd_5.pdf: 12 chunks, 4485 words


Cannot set gray stroke color because /'P3' is an invalid float value
Cannot set gray non-stroke color because /'P3' is an invalid float value
Cannot set gray stroke color because /'P9' is an invalid float value
Cannot set gray non-stroke color because /'P9' is an invalid float value
Cannot set gray stroke color because /'P11' is an invalid float value
Cannot set gray non-stroke color because /'P11' is an invalid float value


ci-cd_8.pdf: 19 chunks, 7239 words
ci-cd_9.pdf: 54 chunks, 21265 words


In [ ]:
'''
openai.api_key = "YOUR_OPENAI_API_KEY"  # replace with your token

# ==========================
# 1) Define a function to generate JSONL from a chunk
# ==========================
def generate_jsonl_from_chunk(chunk_text, filename, section_number):
    """
    Send a chunk of text to the model and get JSON-formatted instruction-output examples.
    """
    prompt = f"""
You are an AI dataset generator for instruction-tuned LLMs.

Text from PDF chunk:
\"\"\"{chunk_text}\"\"\"

Your task:
1. Generate as many instruction-input-output examples as possible (up to 50000 if you can).
2. Output MUST be a valid JSON list of objects.
3. Each object MUST have the keys:
   - "instruction": a question/task based on PDF content
   - "input": context or relevant info from the PDF
   - "output": correct answer/explanation
4. DO NOT include any text outside the JSON list.

Each example's input should reference the filename and section number like:
"From PDF {filename}, section {section_number}"
"""

    response = openai.ChatCompletion.create(
        model="gpt-4",        # or gpt-5 if available
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=2000,      # adjust depending on chunk size
    )

    raw_output = response['choices'][0]['message']['content']

    try:
        data = json.loads(raw_output)
        if isinstance(data, list):
            # Make sure input field contains filename + section
            for ex in data:
                ex["input"] = f"From PDF {filename}, section {section_number}: " + ex.get("input", "")
            return data
        else:
            print(f"⚠️ Unexpected format in section {section_number} of {filename}")
            return [{"instruction": "Parse failure, raw output",
                     "input": f"From PDF {filename}, section {section_number}",
                     "output": raw_output}]
    except json.JSONDecodeError:
        print(f"⚠️ JSON parse failed for section {section_number} of {filename}")
        return [{"instruction": "Parse failure, raw output",
                 "input": f"From PDF {filename}, section {section_number}",
                 "output": raw_output}]

# ==========================
# 2) Loop over pdf_chunks and generate dataset
# ==========================
dataset = []

for fname, chunks in pdf_chunks.items():
    for i, chunk in enumerate(tqdm(chunks, desc=f"Processing {fname}")):
        examples = generate_jsonl_from_chunk(chunk, fname, i+1)
        dataset.extend(examples)

# ==========================
# 3) Save dataset as JSONL
# ==========================
output_file = "ci_cd_dataset.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for ex in dataset:
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")

print(f"✅ Saved {len(dataset)} examples into {output_file}")
'''

In [8]:
dataset = "ci_cd_dataset.jsonl"

### ✅ 3. Embedding Model
- Loaded **MiniLM (all-MiniLM-L6-v2)** for sentence embeddings.
- Converted each `output` into a **vector representation**.
- Stored embeddings + metadata (`instruction`, `output`) in FAISS index.

In [ ]:
'''
# ==========================
# 1) Configs
# ==========================
model_name = "google/flan-t5-small"
dataset_path = "ci_cd_dataset.jsonl"
max_length = 512
batch_size = 2
num_epochs = 3
learning_rate = 2e-4
device = "mps" if torch.backends.mps.is_available() else "cpu"

# ==========================
# 2) Load dataset
# ==========================
dataset = load_dataset("json", data_files=dataset_path)
dataset = dataset["train"].train_test_split(test_size=0.1)

# ==========================
# 3) Load tokenizer
# ==========================
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ==========================
# 4) Load model first
# ==========================
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.to(device)

# ==========================
# 5) Apply LoRA
# ==========================
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],  # T5-specific
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)
model = get_peft_model(model, lora_config)

# ==========================
# 6) Tokenize dataset correctly
# ==========================
def tokenize_fn(examples):
    # Combine instruction and input into a single string for each example
    # Use a simple f-string for clarity and to handle cases with or without input
    inputs = [
        f"{instruction}\n{input_text}" if input_text else instruction
        for instruction, input_text in zip(examples["instruction"], examples["input"])
    ]

    # Tokenize the combined inputs
    tokenized_inputs = tokenizer(
        inputs,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    # Tokenize the outputs to create the labels
    labels = tokenizer(
        examples["output"],
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    
    # Set the labels, ensuring the padding token is ignored during loss calculation
    tokenized_inputs["labels"] = labels["input_ids"]
    tokenized_inputs["labels"][labels["input_ids"] == tokenizer.pad_token_id] = -100

    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_fn, batched=True, remove_columns=["instruction", "input", "output"])

train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]

# ==========================
# 7) DataLoader + DataCollator
# ==========================
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=data_collator)
eval_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=data_collator)

# ==========================
# 8) Optimizer
# ==========================
optimizer = AdamW(model.parameters(), lr=learning_rate)

# ==========================
# 9) Training loop
# ==========================
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        # Move all tensors to device
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {total_loss/len(train_loader):.4f}")

# ==========================
# 10) Save LoRA-adapted model
# ==========================
model.save_pretrained("./flan_t5_lora_mac")
tokenizer.save_pretrained("./flan_t5_lora_mac")

# ==========================
# Inference Cell (works now)
# ==========================
# Config
model_path = "./flan_t5_lora_mac"  # where you saved after training
device = "mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)

# ==========================
# 1) Standardized Prompt Builder
# ==========================
def build_prompt(user_prompt: str, retrieved_context: str = "") -> str:
    if retrieved_context and retrieved_context.strip():
        return f"""You are an AI assistant with prior knowledge from your training. 
You also have access to external information retrieved from a knowledge base (vector DB). 

Use BOTH:
1. Your trained knowledge  
2. The retrieved context below  

to provide the best, most accurate, and helpful response to the user’s question.

User Question:
{user_prompt}

Retrieved Context:
{retrieved_context}

Final Answer:"""
    else:
        return f"""You are an AI assistant with prior knowledge from your training. 

Answer the following question as best as possible using your trained knowledge.

User Question:
{user_prompt}

Final Answer:"""

# ==========================
# 2) Generate Response Function
# ==========================
def generate_response(user_prompt: str, retrieved_context: str = "", max_new_tokens: int = 200) -> str:
    prompt = build_prompt(user_prompt, retrieved_context)
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=0.9,
        temperature=0.7
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ==========================
# 3) Example Usage
# ==========================
#retrieved_context = the chunk(s) of text you pulled from your vector DB (e.g., FAISS, Pinecone, Chroma) that are most relevant to the user’s query.
#user_prompt = the actual question or instruction the user gave.

retrieved_context = """
CI/CD (Continuous Integration and Continuous Deployment) is a DevOps practice 
that helps automate building, testing, and deploying applications. 
In machine learning (MLOps), CI/CD ensures models are retrained, validated, 
and deployed to production in a reliable and repeatable way.
"""

# Example usage with your generate_response function
print(generate_response(
    "Explain CI/CD for machine learning in simple words.", 
    retrieved_context
))

print(generate_response(
    "Translate this into French:", 
    "Continuous Integration and Continuous Deployment (CI/CD) helps automate testing and deployment for ML pipelines."
))
'''

### ✅ 4. Vector Database Creation and Retrieval Function
- Built a FAISS index for efficient similarity search.
- Metadata is stored in parallel to retrieve the **exact output text** when queried.
- A function retrieves **top-k similar outputs** from FAISS given a new query.
- Ensures semantic similarity, not just keyword matching.

In [3]:
jsonl_file = "ci_cd_dataset.jsonl"   # your dataset file
outputs = []
with open(jsonl_file, "r") as f:
    for line in f:
        obj = json.loads(line)
        outputs.append(obj["output"])   # only take output

In [4]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = embedder.encode(outputs, convert_to_numpy=True)

dimension = embeddings.shape[1]  # embedding size
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

id_to_output = {i: outputs[i] for i in range(len(outputs))}

def query_vector_db(query, k=2):
    query_emb = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_emb, k)
    return [id_to_output[i] for i in indices[0]]

In [23]:
# --- Example usage ---
query = "hat are the benefits of using Docker in CI/CD?"
results = query_vector_db(query, k=2)

print("Top results from vector DB:")
for r in results:
    print("-", r)

Top results from vector DB:
- 1. Rapid setup of development environments. 2. Easy deployment in cloud or on-premises. 3. Less risky releases due to standardized environments. 4. Improved testing and QA. 5. Seamless continuous deployment to staging and production.
- 1. Rapid setup of development environments. 2. Easy deployment in cloud or on-premises. 3. Less risky releases due to standardized environments. 4. Improved testing and QA. 5. Seamless continuous deployment to staging and production.


### ✅ 5. LLM Integration (RAG) and Answer Generation
- Loaded **FLAN-T5-small (~300MB)** as the reasoning model.
- Constructed a **prompt** combining:
  - User query  
  - Retrieved context from vector DB  
  - Explicit instructions on how to refine, optimize, or correct the answer.
- If Vector DB contains relevant info → LLM optimizes and explains it.  
- If Vector DB info is incomplete → LLM generates a correct, standalone answer.

In [18]:
device = "mps" if torch.backends.mps.is_available() else "cpu"

llm_model_name = "google/flan-t5-small"
llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(llm_model_name)
llm_model.to(device)
llm_model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [19]:
def retrieve_from_vector_db(query, embedder, index, id_to_output, k=3):
    query_emb = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_emb, k)
    results = [id_to_output[i] for i in indices[0]]
    return results

In [81]:
def answer_with_rag(query, embedder, index, id_to_output, llm_model, llm_tokenizer, k=3):
    # Retrieve relevant "outputs" from vector DB
    retrieved_outputs = retrieve_from_vector_db(query, embedder, index, id_to_output, k=k)
    context = "\n".join(retrieved_outputs)

    # Create a prompt
    prompt = f"""
You are an expert assistant.

User query:
{query}

Relevant information retrieved from the knowledge base:
{context}

Instructions:
- If the context already fully answers the query, summarize it concisely.
- If the context is partial, improve/complete it with your own knowledge.
- Keep the answer clear and professional.

Final Answer:
"""

    # Generate answer
    inputs = llm_tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = llm_model.generate(**inputs, max_new_tokens=200)
    answer = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer, retrieved_outputs[0]

In [106]:
def ci_cd_fintuned_llm_vectordb_output(user_query = ''):
    llm_response, context = answer_with_rag(
        user_query,
        embedder,          # the sentence-transformers model
        index,
        id_to_output,
        llm_model,
        llm_tokenizer,
        k=2
    )
    
    '''
    print("LLM Response:")
    print(llm_response)
    print('\n')
    print("Vector DB Response:")
    print(vectordb_response)
    '''
    print("LLM + VectorDB Response:")
    print(context)

### ✅ 6. Testing
Example queries tested:
- *“What is CI in Agile?”*  
- *“Benefits of Docker in CI/CD?”*  
- *“How does Kubernetes handle scaling?”*  

In [107]:
ci_cd_fintuned_llm_vectordb_output(user_query = 'What is the role of Continuous Integration (CI) in Agile development?')

LLM + VectorDB Response:
Continuous Integration (CI) originated from Extreme Programming (XP). It is an Agile method where team members regularly integrate code changes into a shared repository. CI enables faster production, improved product quality, and better team effectiveness by automating testing and development.


In [108]:
ci_cd_fintuned_llm_vectordb_output(user_query = 'How can CI/CD pipelines be secured in a cloud environment?')

LLM + VectorDB Response:
In data engineering, CI/CD pipelines automate end-to-end workflows including data ingestion, transformation, validation, and storage. This ensures consistent data quality, reduces risks from manual deployments, and accelerates analytics and model deployment.


In [109]:
ci_cd_fintuned_llm_vectordb_output(user_query = 'What are the main benefits of using Docker in CI/CD workflows?')

LLM + VectorDB Response:
In data engineering, CI/CD pipelines automate end-to-end workflows including data ingestion, transformation, validation, and storage. This ensures consistent data quality, reduces risks from manual deployments, and accelerates analytics and model deployment.


In [110]:
ci_cd_fintuned_llm_vectordb_output(user_query = 'How does Kubernetes Deployment help with application scaling?')

LLM + VectorDB Response:
Kubernetes Deployments manage Replica Sets, which control pod scaling. They allow configuration changes, rolling updates, and automated rollback if issues occur, ensuring stable scaling and quick recovery.


In [111]:
ci_cd_fintuned_llm_vectordb_output(user_query = 'Explain the concept of a CI/CD pipeline in data engineering.')

LLM + VectorDB Response:
In data engineering, CI/CD pipelines automate end-to-end workflows including data ingestion, transformation, validation, and storage. This ensures consistent data quality, reduces risks from manual deployments, and accelerates analytics and model deployment.


In [112]:
ci_cd_fintuned_llm_vectordb_output(user_query = 'What are common challenges in automating testing within CI/CD pipelines?')

LLM + VectorDB Response:
In data engineering, CI/CD pipelines automate end-to-end workflows including data ingestion, transformation, validation, and storage. This ensures consistent data quality, reduces risks from manual deployments, and accelerates analytics and model deployment.


In [113]:
ci_cd_fintuned_llm_vectordb_output(user_query = 'How do you implement rolling updates in Kubernetes?')

LLM + VectorDB Response:
Kubernetes Deployments manage Replica Sets, which control pod scaling. They allow configuration changes, rolling updates, and automated rollback if issues occur, ensuring stable scaling and quick recovery.


In [114]:
ci_cd_fintuned_llm_vectordb_output(user_query = 'What is the difference between Continuous Delivery and Continuous Deployment?')

LLM + VectorDB Response:
1. Rapid setup of development environments. 2. Easy deployment in cloud or on-premises. 3. Less risky releases due to standardized environments. 4. Improved testing and QA. 5. Seamless continuous deployment to staging and production.


In [115]:
ci_cd_fintuned_llm_vectordb_output(user_query = 'How can Docker improve reproducibility in development environments?')

LLM + VectorDB Response:
1. Rapid setup of development environments. 2. Easy deployment in cloud or on-premises. 3. Less risky releases due to standardized environments. 4. Improved testing and QA. 5. Seamless continuous deployment to staging and production.


In [116]:
ci_cd_fintuned_llm_vectordb_output(user_query = 'What monitoring tools are recommended for CI/CD pipelines in production?')

LLM + VectorDB Response:
In data engineering, CI/CD pipelines automate end-to-end workflows including data ingestion, transformation, validation, and storage. This ensures consistent data quality, reduces risks from manual deployments, and accelerates analytics and model deployment.


## ✅ 7. Key Features
The pipeline successfully retrieved relevant outputs and refined them using the LLM.
- **Lightweight** → uses FLAN-T5-small instead of large models.  
- **Domain-Specific** → trained on CI/CD, Docker, Kubernetes knowledge.  
- **Flexible** → easy to extend by adding more JSONL data.  
- **RAG-based** → ensures answers are both retrieved and refined. 

## ✅ 8. 🏁 Conclusion

This project successfully demonstrates the implementation of a domain-specific **Retrieval-Augmented Generation (RAG) pipeline** for DevOps knowledge retrieval. By combining a lightweight **LLM (`google/flan-t5-small`)** with a **FAISS vector database**, we've created an efficient and highly-relevant question-answering system.

The key achievements of this project include:

* **Effective Knowledge Retrieval**: The pipeline effectively retrieves relevant information from the curated dataset, addressing the challenges of providing accurate and context-aware answers.
* **Performance with Lightweight Models**: We've shown that high performance can be achieved without relying on a large, resource-intensive LLM, making the solution more accessible and cost-effective.
* **End-to-End Workflow**: This notebook provides a complete and reproducible workflow, from data preparation and vectorization to the final RAG query and response.

### Future Work

* **Expanding the Dataset**: Integrate a wider range of documentation and knowledge sources to expand the system's expertise.
* **Advanced Evaluation**: Implement more rigorous evaluation metrics to benchmark the model's accuracy and relevance against other RAG systems.
* **Deployment**: Package the pipeline into a simple API for easy integration into web applications or chatbots.